# Unit Test Downloader and Metadata Extractor
This notebook collects the commands to download a version of the unit test models from the SysML reference implementation API site.

## Download Data from the Server

Contact the API server and pull down relevant data.

In [ ]:
import json
from pathlib import Path

from pymbe.local.bundle import download_to_local

import pymbe.api as pm
from pymbe.client import SysML2Client
from pymbe.label import *
from pymbe.graph.lpg import SysML2LabeledPropertyGraph
from pymbe.interpretation.calc_dependencies import *

To configure the statement below, look for the project and commit ids at . Then put your desired download directory below. Forward slashes work fine, even on Windows.

In [ ]:
fixtures = Path("..") / "tests" / "fixtures"

In [ ]:
download_to_local(
    "http://sysml2.intercax.com",
    "Kerbal.json",
    "2141dcd2-33b5-4ca7-b284-8ff4bb150b52",
    "de71ca5b-8a53-4fb2-a0a3-afde48b04aba",
    str(fixtures.absolute()),
)

Inspect the data that have been downloaded (don't load into client yet).

## Load the Data and Generate a Stable Name and ID Map

In [ ]:
helper_client = SysML2Client()

elements_data_path = Path("..") / "tests" / "fixtures" / "Kerbal.json"
assert elements_data_path.exists(), f"Could not find: '{elements_data_path}'"

helper_client._load_from_file(elements_data_path)

lpg = SysML2LabeledPropertyGraph()
lpg.update(helper_client.elements_by_id, False)

exec_order = generate_execution_order(lpg)
sig_map = generate_parameter_signature_map(helper_client.elements_by_id, exec_order)

id_to_qualified_name = {}
qualified_name_to_id = {}

In [ ]:
for ele_id, node in dict(lpg.nodes).items():
    qualified_label = get_qualified_label(node, helper_client.elements_by_id, sig_map, True)
    id_to_qualified_name.update({ele_id: qualified_label})
    if qualified_label in qualified_name_to_id:
        print(f"Duplicate name found for {qualified_label}")
    
    qualified_name_to_id.update({qualified_label: ele_id})

In [ ]:
len(qualified_name_to_id.items())

In [ ]:
len(id_to_qualified_name.items())

In [ ]:
qualified_name_to_id